In [ ]:
import json
import matplotlib.pyplot as plt

In [ ]:
def GetData(file):
    '''
    Parses the data in a `.json` file produced by the Unity simulation into an array for each body

    Returns:
        `timeValues`: Array of time values

        `celestialBodies`: Dictionary of arrays for each celestial body

        `lagrangePoints`: Dictionary of arrays for each lagrange point

        `barycentres` : Dictionary of arrays for each barycentre
    '''
    raw = json.load(file)

    timeValues = []
    celestialBodies = {}
    lagrangePoints = {}
    barycentres = {}
    for entry in raw:
        timeValues.append(entry['time'])
        celestialBodyRaw = entry['celestialBodyData']
        for cb in celestialBodyRaw:
            name = cb['name']
            if (name not in celestialBodies):
                celestialBodies[name] = []
            celestialBodies[name].append(cb)

        lagrangePointRaw = entry['lagrangePointData']
        for lp in lagrangePointRaw:
            name = lp['name']
            if (name not in lagrangePoints):
                lagrangePoints[name] = []
            lagrangePoints[name].append(lp)

        barycentresRaw = entry['barycentreData']
        for b in barycentresRaw:
            name = b['name']
            if (name not in barycentres):
                barycentres[name] = []
            barycentres[name].append(b)

    return timeValues, celestialBodies, lagrangePoints, barycentres

In [ ]:
def GetPositions(arrayOfDictsWithPos):
    '''
    Creates a dictionary of the positions of a celestial body or lagrange point through time
    '''
    positions = {
        'x' : [e['position']['x'] for e in arrayOfDictsWithPos],
        'y' : [e['position']['y'] for e in arrayOfDictsWithPos],
        'z' : [e['position']['z'] for e in arrayOfDictsWithPos]
    }
    return positions

In [ ]:
def TranslateAllPositions(positionsDict, translations):
    '''
    Translates the positions in positionsDict by subtracting the positions in translations
    '''
    translatedPositions = {
        'x': [],
        'y': [],
        'z': []
    }
    for i in range(len(positionsDict['x'])):
        translatedPositions['x'].append(positionsDict['x'][i] - translations['x'][i])
        translatedPositions['y'].append(positionsDict['y'][i] - translations['y'][i])
        translatedPositions['z'].append(positionsDict['z'][i] - translations['z'][i])

    return translatedPositions

In [ ]:
def PlotPositions(positions, label):
    startX = positions['x'][0]
    startZ = positions['z'][0]
    plt.scatter(startX, startZ)
    plt.plot(positions['x'], positions['z'], label=label)

In [ ]:
f = open('Data/Basic_Setup.json')
timeValues, celestialBodies, lagrangePoints, barycentres = GetData(f)
f.close()

# Get positions
jupiterPos = GetPositions(celestialBodies['Jupiter'])
leadingObjectPos = GetPositions(celestialBodies['Leading Object'])
sunPos = GetPositions(celestialBodies['Sun'])
baryPos = GetPositions(barycentres['Barycentre'])
L4Pos = GetPositions(lagrangePoints['Sun-Jupiter L4'])

# Normalize all positions around the barycentre
jupiterPos = TranslateAllPositions(jupiterPos, baryPos)
leadingObjectPos = TranslateAllPositions(leadingObjectPos, baryPos)
sunPos = TranslateAllPositions(sunPos, baryPos)
L4Pos = TranslateAllPositions(L4Pos, baryPos)

fig = plt.figure(figsize=(5,5))
# PlotPositions(jupiterPos, 'Jupiter')
PlotPositions(L4Pos, 'L4')
PlotPositions(leadingObjectPos, 'Leading Object')
PlotPositions(sunPos, 'Sun')
# PlotPositions(baryPos, 'barycentre')
plt.grid()
plt.legend()
plt.xlabel('x')
plt.ylabel('z', rotation=0)
plt.title('The normalized orbit of the L4 point \nand the asteroid without offset')
fig.show()

In [ ]:
# Plots all mass offset orbits in one plot
# Data file names
massOffsetNames = ['Data/Mass_Offset_10.json', 'Data/Mass_Offset_100.json', 'Data/Mass_Offset_500.json', 'Data/Mass_Offset_1000.json']
# Names for labels in plot
massOffsetLabels = ['10', '100', '500', '1000']
isFirst = True

fig = plt.figure(figsize=(5,5))

labelIdx = 0
for name in massOffsetNames:
    f = open(name)
    timeValues, celestialBodies, lagrangePoints, barycentres = GetData(f)
    f.close()

    # Get positions of relevant bodies
    leadingObjectPos = GetPositions(celestialBodies['Leading Object'])
    sunPos = GetPositions(celestialBodies['Sun'])
    L4Pos = GetPositions(lagrangePoints['Sun-Jupiter L4'])
    baryPos = GetPositions(barycentres['Barycentre'])

    # Normalize positions around barycentre
    leadingObjectPos = TranslateAllPositions(leadingObjectPos, baryPos)
    sunPos = TranslateAllPositions(sunPos, baryPos)
    L4Pos = TranslateAllPositions(L4Pos, baryPos)

    # Plot L4 only once
    if isFirst:
        PlotPositions(L4Pos, 'Sun-Jupiter L4')
        isFirst = False


    PlotPositions(leadingObjectPos, massOffsetLabels[labelIdx])
    labelIdx+=1


plt.grid()
plt.legend()
plt.xlabel('x')
plt.ylabel('z', rotation=0)
plt.title('The normalized orbits of asteroids with different offsets in mass')
fig.show()


In [ ]:
# Set this for offset in X direction True) or Z direction (False)
useXOffset = False

# Plots all X position offsets in one plot
# Data file names
posXOffsetNames = ['Data/Position_Offset_(-30,0,0).json', 'Data/Position_Offset_(-20,0,0).json', 'Data/Position_Offset_(-10,0,0).json', 'Data/Position_Offset_(10,0,0).json', 'Data/Position_Offset_(20,0,0).json', 'Data/Position_Offset_(30,0,0).json']
posZOffsetNames = ['Data/Position_Offset_(0,0,-30).json', 'Data/Position_Offset_(0,0,-20).json', 'Data/Position_Offset_(0,0,-10).json', 'Data/Position_Offset_(0,0,10).json', 'Data/Position_Offset_(0,0,20).json', 'Data/Position_Offset_(0,0,30).json']
# Names for labels in plot
plotLabels = ['-30', '-20', '-10', '10', '20', '30']
isFirst = True

if useXOffset:
    fileNames = posXOffsetNames
    plotTitle = "The normalized orbits of asteroids with \ndifferent position offsets in the X direction"
else:
    fileNames = posZOffsetNames
    plotTitle = "The normalized orbits of asteroids with \ndifferent position offsets in the Z direction"

fig = plt.figure(figsize=(5,5))

labelIdx = 0
for name in fileNames:
    f = open(name)
    timeValues, celestialBodies, lagrangePoints, barycentres = GetData(f)
    f.close()

    # Get positions of relevant bodies
    leadingObjectPos = GetPositions(celestialBodies['Leading Object'])
    sunPos = GetPositions(celestialBodies['Sun'])
    L4Pos = GetPositions(lagrangePoints['Sun-Jupiter L4'])
    baryPos = GetPositions(barycentres['Barycentre'])

    # Normalize positions around barycentre
    leadingObjectPos = TranslateAllPositions(leadingObjectPos, baryPos)
    sunPos = TranslateAllPositions(sunPos, baryPos)
    L4Pos = TranslateAllPositions(L4Pos, baryPos)

    # Plot L4 only once
    if isFirst:
        PlotPositions(L4Pos, 'Sun-Jupiter L4')
        isFirst = False


    PlotPositions(leadingObjectPos, plotLabels[labelIdx])
    labelIdx+=1

plt.grid()
plt.legend()
plt.title(plotTitle)
plt.xlabel("x")
plt.ylabel("z", rotation=0)
fig.show()


In [ ]:
# Set this for offset in X direction True) or Z direction (False)
useXOffset = False

# Plots all X velocity offsets in one plot
# Data file names
velocityXOffsetNames = ['Data/Velocity_Offset_(-30,0,0).json', 'Data/Velocity_Offset_(-20,0,0).json', 'Data/Velocity_Offset_(-10,0,0).json', 'Data/Velocity_Offset_(10,0,0).json', 'Data/Velocity_Offset_(20,0,0).json', 'Data/Velocity_Offset_(30,0,0).json']
velocityZOffsetNames = ['Data/Velocity_Offset_(0,0,-30).json', 'Data/Velocity_Offset_(0,0,-20).json', 'Data/Velocity_Offset_(0,0,-10).json', 'Data/Velocity_Offset_(0,0,10).json', 'Data/Velocity_Offset_(0,0,20).json', 'Data/Velocity_Offset_(0,0,30).json']
# Names for labels in plot
plotLabels = ['-30', '-20', '-10', '10', '20', '30']
isFirst = True

if useXOffset:
    fileNames = velocityXOffsetNames
    plotTitle = "The normalized orbits of asteroids with \ndifferent velocity offsets in the X direction"
else:
    fileNames = velocityZOffsetNames
    plotTitle = "The normalized orbits of asteroids with \ndifferent velocity offsets in the Z direction"

fig = plt.figure(figsize=(5,5))

labelIdx = 0
for name in fileNames:
    f = open(name)
    timeValues, celestialBodies, lagrangePoints, barycentres = GetData(f)
    f.close()

    # Get positions of relevant bodies
    leadingObjectPos = GetPositions(celestialBodies['Leading Object'])
    sunPos = GetPositions(celestialBodies['Sun'])
    L4Pos = GetPositions(lagrangePoints['Sun-Jupiter L4'])
    baryPos = GetPositions(barycentres['Barycentre'])

    # Normalize positions around barycentre
    leadingObjectPos = TranslateAllPositions(leadingObjectPos, baryPos)
    sunPos = TranslateAllPositions(sunPos, baryPos)
    L4Pos = TranslateAllPositions(L4Pos, baryPos)

    # Plot L4 only once
    if isFirst:
        PlotPositions(L4Pos, 'Sun-Jupiter L4')
        isFirst = False


    PlotPositions(leadingObjectPos, plotLabels[labelIdx])
    labelIdx+=1


plt.grid()
plt.legend()
plt.xlabel('x')
plt.ylabel('z', rotation=0)
plt.title(plotTitle)
fig.show()


In [ ]:
# TODO: Maybe show influence of mass offset on sun